# Impermanent Loss Agent

The `ImpermanentLossAgent` monitors a when value of investment falls below a certain threshold using ``UniswapImpLoss`` object. It integrates DeFiPy’s LPQuote for exits and Web3Scout’s event feeds for real-time updates, supporting off-chain testing and on-chain execution.

### 📘 Notable Classes

---

* **Class**: 📘 ``defipy.agents.config.ImpermanentLossConfig`` 
    * **Purpose**: Configuration for volume threshold agent.
        * **Parameters**:
            * `volume_threshold`: Volume threshold to check (`float`).
            * `pool_address`: UniV2 pool address (`str`).
            * `provider_url`: Provider URL (eg, infura.io)(`str`).
            * `platform`: Platform where pool resides (eg, uniswap) (`str`).
            * `abi_name`: ABI handle pointing to pool contract in JSON (eg, UniswapV2Pair) (`str`).

---
         
* **Class**: 📘 ``defipy.agents.ImpermanentLossAgent`` 
    * **Purpose**: Reactive DeFi agent for determining price threshold.
        * **Parameters**:
            * `config`: Agent configuration parameters (`PriceThresholdConfig`).
    * **Methods**:
        * ``apply()``
            * **Purpose**: Apply price threshold agent
        * ``run_batch(lp: UniswapExchange, tkn: ERC20, user_nm: str, events: dict)``
            * **Purpose**: Run AI price agent on batch data 
            * **Parameters**:
                * `lp`: Swap token (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `user_nm`: Account name (`str`).
                * `events`: Dictionary of sync events (`dict`).
        * ``apply(lp: UniswapExchange, tkn: ERC20, user_nm: str, block_num: int)``
            * **Purpose**: Apply TVL check
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `user_nm`: Account name (`str`).
                * `block_num`: Block number (`int`).          
        * ``check_condition(tkn: ERC20, threshold: float)``
            * **Purpose**: Check if position is below threshold  
            * **Parameters**:
                * `tkn`: Swap token (`ERC20`).
                * `threshold`: override config price threshold (optional) (`float`).
        * ``get_current_position_value(tkn: ERC20)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `tkn`: Swap token (`ERC20`).
        * ``take_mock_position(lp: UniswapExchange, tkn: ERC20, user_nm: str, amt: float)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `user_nm`: Account name (`str`).
                * `amt`: Mock token amount (`float`).
        * ``withdraw_mock_position(lp: UniswapExchange, tkn: ERC20, user_nm: str, lp_amt: float)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `user_nm`: Account name (`str`).
                * `lp_amt`: Mock LP amount (`float`).
        * ``update_mock_pool(lp: UniswapExchange, cur_block: int)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `cur_block`: Current block number (`int`).
        * ``prime_mock_pool(start_block: int, user_nm: str)``
            * **Purpose**: Initialize off-chain pool using information in start_block
            * **Parameters**:
                * `start_block`: Start block (`int`).
                * `user_nm`: Account name of mock off-chain pool (`str`).
        * ``get_impermanent_loss()``
            * **Purpose**: Getter function for impermanent loss of position measured in (%)
        * ``def get_current_position_value()``
            * **Purpose**: Getter function for current position value of investment
        * ``get_w3()``
            * **Purpose**: Getter function for web3.py connector object
        * ``get_abi()``
            * **Purpose**: Getter function for ABI json data
        * ``get_contract_instance()``
            * **Purpose**: Getter function for pool contract instance
        * ``get_lp_data()``
            * **Purpose**: Getter function for pool data associated with `pool_address` from config
        * ``def get_iloss()``
            * **Purpose**: Getter function for ImpLoss object.


In [1]:
from defipy import *
from web3scout import *

In [2]:
il_threshold = 99.70
pair_address = "0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc"
provider_url = "https://mainnet.infura.io/v3/xxxxxxxxxxxxx"
platform = Platform.AGNOSTIC
abi_name = JSONContract.UniswapV2Pair
user_position = 100
exit_percentage = 5

config = ImpermanentLossConfig(
    il_threshold = il_threshold,
    pool_address = pair_address,
    provider_url = provider_url,
    platform = platform,
    abi_name = abi_name,
    user_position = user_position,
    exit_percentage = exit_percentage
)

agent = ImpermanentLossAgent(config)
agent.init()

print(f"Monitoring TVL changes @ pool address {pair_address}")

Monitoring TVL changes @ pool address 0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc


In [3]:
abi = ABILoad(platform, abi_name)
connect = ConnectW3(provider_url)
connect.apply()

last_block = connect.get_w3().eth.block_number
start_block = last_block - 100

# Grab batch sync events from pool
rEvents = RetrieveEvents(connect, abi)
events = rEvents.apply(EventType.SWAP, address = pair_address, start_block=start_block, end_block=last_block)
df_events = rEvents.to_dataframe(events)
df_events.head(2)

,blockNumber,event,address,blockHash,logIndex,transactionHash,transactionIndex,args
0,23465266,Swap,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,0xdcddce03378ef0979a7324268bc434f17ab26496466f...,596,0xa7c3a5cbd3792cda4c8b75fe36b9e8f97aabbe169c10...,155,{'sender': '0x5141B82f5fFDa4c6fE1E372978F1C542...
1,23465297,Swap,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,0x9222687c4e46e75da9ffa9b0a6272cfbcb4bb89a6ceb...,288,0xf986f4b7cf60f36502b9f2d424db4398bfc736712d89...,64,{'sender': '0x7a250d5630B4cF539739dF2C5dAcb4c6...


### Agent Run #1

In [4]:
tkn0 = agent.get_lp_data().tkn0
tkn1 = agent.get_lp_data().tkn1
lp = agent.prime_mock_pool(start_block, 'user')
lp.summary()

Exchange USDC-WETH (LP)
Reserves: USDC = 14160246.665848, WETH = 3424.8649425013764
Liquidity: 0.08143220235298784 



In [5]:
# Take a position
lp_pos = agent.take_mock_position(lp, tkn0, 'user', 100)

In [6]:
# Run impermanent loss analysis
agent.run_batch(lp, tkn0, 'user', events)

Block 23465266: Value (USDC) = 99.6996420512981, outside loss threshold 99.7
Block 23465297: Value (USDC) = 99.69965483382371, outside loss threshold 99.7
Block 23465299: Value (USDC) = 99.69966645871294, outside loss threshold 99.7
Block 23465301: Value (USDC) = 99.69967698120095, outside loss threshold 99.7
Block 23465305: Value threshold condition met for USDC-WETH LP
Block 23465307: Value threshold condition met for USDC-WETH LP
Block 23465319: Value threshold condition met for USDC-WETH LP
Block 23465319: Value threshold condition met for USDC-WETH LP
Block 23465326: Value threshold condition met for USDC-WETH LP
Block 23465333: Value threshold condition met for USDC-WETH LP
Block 23465337: Value threshold condition met for USDC-WETH LP
